<a href="https://colab.research.google.com/github/melrahmtz/purple-box/blob/main/hand-on-practice/vectordb_supabase/1602_vector_embedding_supabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv --quiet
!pip install langchain --quiet
!pip install supabase --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet
!pip install numpy --quiet
!pip install transformers --quiet



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [ ]:
import os
import numpy
from supabase.client import Client, create_client
from langchain.vectorstores import SupabaseVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

openai_key  = os.environ['OPENAI_API_KEY']
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownTextSplitter

# Load the Markdown file
markdown_path = "PDF1(updated).md"  # Replace with your file path
loader = TextLoader(markdown_path)
documents = loader.load()

# Split the document into chunks using MarkdownTextSplitter
text_splitter = MarkdownTextSplitter(chunk_size=2048, chunk_overlap=600) # Experience with the chunk_size and chunk_overlap
docs = text_splitter.split_documents(documents)

print(f"Loaded {len(docs)} chunks")


Loaded 40 chunks


In [ ]:
vector_store = SupabaseVectorStore.from_documents(
    docs,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=500,
)

In [ ]:
#query = "What is Sentiment Analysis"
#query = "What is the accuracy of the LDA model when using the \"daily weighted average\" sentiment score?"
query = "What is the characteristic of the sentiment score distribution?"

matched_docs = vector_store.similarity_search(query)
print(matched_docs[0].page_content)

The latter sections present a more sophisticated method of processing the sentiment score, with supporting results provided.

Figure 1 presents a sample distribution of calculated sentiment scores, showing a notable negative skew.

Figure 1: Distribution of Sentiment Scores

![Image](/content/markdown/PDF1_artifacts/image_000000_76f7bf71b5ee016569038b353d497dbab4ece2118c183a487e21be8cd18ca35c.png)

*Image Description:* The image shows a histogram with the title "Distribution of Sentiment Scores." The x-axis represents sentiment scores, ranging from -1.0 to 1.0, with a range of 1,000 points each. The y-axis shows the frequency of sentiment scores, ranging from -1,000 to 0, with a scale from 0 to 1,985, which indicates a total of 1,885 points. 

A prominent blue bar extends vertically most heavily, touching the sentiment score of 0.000, indicating it is the most frequent sentiment score in this histogram. Other smaller bars correspond to various sentiment scores, some reaching up to 0.00

In [ ]:
# Similarity Search with Scores

matched_docs = vector_store.similarity_search_with_relevance_scores(query)
matched_docs[0]

(Document(metadata={'source': 'PDF1(updated).md'}, page_content='The latter sections present a more sophisticated method of processing the sentiment score, with supporting results provided.\n\nFigure 1 presents a sample distribution of calculated sentiment scores, showing a notable negative skew.\n\nFigure 1: Distribution of Sentiment Scores\n\n![Image](/content/markdown/PDF1_artifacts/image_000000_76f7bf71b5ee016569038b353d497dbab4ece2118c183a487e21be8cd18ca35c.png)\n\n*Image Description:* The image shows a histogram with the title "Distribution of Sentiment Scores." The x-axis represents sentiment scores, ranging from -1.0 to 1.0, with a range of 1,000 points each. The y-axis shows the frequency of sentiment scores, ranging from -1,000 to 0, with a scale from 0 to 1,985, which indicates a total of 1,885 points. \n\nA prominent blue bar extends vertically most heavily, touching the sentiment score of 0.000, indicating it is the most frequent sentiment score in this histogram. Other sm

In [ ]:
query_embedding = embeddings.embed_query("Test query")
print(f"Query embedding size: {len(query_embedding)}")


Query embedding size: 384


In [ ]:
# SupabaseVectorStore as a retriever

retriever = vector_store.as_retriever(search_type="mmr")
matched_docs = retriever.invoke(query)

for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)

ValueError: Number of columns in X and Y must be the same. X has shape (1, 384) and Y has shape (20, 0).